In [ ]:
# default_exp extract

# Data Preprocessing

> The TMDB dataset is only accessible through an API. Hence, we will write a few functions to extract the dataset, perform some simple preprocessing before saving to csv files.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

We first extract the dataset by accessing the TMDB API. Since not all requests will be successful, a larger than desired dataset size value should be used to return an eventual dataset size of roughly the same magnitude. 

In [ ]:
#exporti

from pandas import DataFrame
from sklearn.utils import shuffle
import os
import concurrent
import requests
import pandas as pd

def extract_dataset(api_key: str, n: int):
    req_url = f"https://api.themoviedb.org/3/movie/{n}?api_key={api_key}&language=en-US"
    response = requests.get(req_url)
    if response.status_code == 200:
        array = response.json()
        return array
    return

In [ ]:
#export

def extract_dataset_threaded(api_key: str, 
                             max_ds_size: int, 
                             max_threads: int) -> DataFrame:
    
    max_threads = max_threads if max_threads < max_ds_size else ds_size
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        movies = [ex for n in range(max_ds_size) if (ex := executor.submit(extract_dataset, api_key, n).result()) is not None]
    return pd.DataFrame.from_records(movies)

Let's run the data extraction functions. 

In [ ]:
#hide
api_key = "785475a69770b7dc1af964feff948dd7"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 6)
pd.set_option('display.width', None)

In [ ]:
api_key = "your_api_key_here"
max_ds_size = 1000
max_threads = 1000
movies = extract_dataset_threaded(api_key=api_key,
                                  max_ds_size=max_ds_size,
                                  max_threads=max_threads)
print(f"Total number of examples downloaded is: {len(movies)}")
movies.head()

Total number of examples downloaded is: 0


""


Looking at the first 5 rows of the scraped dataset, we easily see that some example taglines (under the tagline column) are missing. We'll need to separate these examples into a separate csv file as they do not have labels. Additionally, we can create our train, validation and test datasets concurrently and save them into separate csv files. This helps reproducibility later on. We also print the relative proportions of each dataset to see if we will need to redo the extraction process above. 

In [ ]:
#export

def create_splits(df: DataFrame,
                  label: str,
                  splits: list,
                  seed: int,
                  keep_missing: bool,
                  save_path: str = "."):

    assert len(splits) == 2, "Train, validation and test splits must be provided, please provide 2 of them as fractions."
    if keep_missing:
        unlabelled_df = df[df[label] == '']
        unlabelled_df.to_csv(os.path.join(save_path, "tagless.csv"))
        print(f"Tagless set size: {len(unlabelled_df)}")
        print("Tagless dataset created!")
    labelled_df = df[df.tagline != '']
    df_size = len(labelled_df)
    labelled_df = shuffle(labelled_df, random_state=seed)
    labelled_df.reset_index(drop=True, inplace=True)
    valid_start, test_start = int(df_size*splits[0]), int(df_size*splits[0] + df_size*splits[1])
    train_df = labelled_df.iloc[:valid_start]
    valid_df = labelled_df.iloc[valid_start:test_start]
    test_df = labelled_df[test_start:]
    print(f"Train set size: {len(train_df)}\nValid set size: {len(valid_df)}\nTest set size: {len(test_df)}")
    train_df.to_csv(os.path.join(save_path, "train.csv"))
    valid_df.to_csv(os.path.join(save_path, "valid.csv"))
    test_df.to_csv(os.path.join(save_path, "test.csv"))
    print("Train, Validation and Test datasets created!")

In [ ]:
splits = [0.7, 0.15]
label = "tagline"
seed = 42
create_splits(df=movies,
              label=label,
              splits=splits,
              seed=seed,
              keep_missing=True)